In [8]:
#Twitter tennis
#Dynamic Graph Temporal Signals
import torch
from torch_geometric_temporal.dataset import TwitterTennisDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader=TwitterTennisDatasetLoader()
dataset=loader.get_dataset()

In [9]:
#Split
train,test=temporal_signal_split(dataset,train_ratio=0.5)

In [10]:
#Model
#Gatted Recurrent Unit
from torch_geometric_temporal.nn.recurrent import GConvGRU
from torch.nn import Module,Linear

class DynamicTemporalNetwork(Module):
    def __init__(self,features,filters):
        super(DynamicTemporalNetwork,self).__init__()

        self.conv_gru=GConvGRU(in_channels=features,out_channels=filters,K=3,normalization='sym')
        self.linear=Linear(in_features=filters,out_features=1)
    
    def forward(self,x,edge_index,edge_weight):
        x=self.conv_gru(x,edge_index,edge_weight).relu()
        x=self.linear(x)

        return x


In [11]:
from torch.nn import MSELoss
from tqdm import tqdm

model=DynamicTemporalNetwork(features=16,filters=32)
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.001,betas=(0.9,0.999))
loss=MSELoss()

In [12]:
#Training
model.train(True)

NUM_EPOCHS=50

for epoch in tqdm(range(NUM_EPOCHS)):
    train_loss=0
    test_loss=0
    #Train
    for t,snap in enumerate(train):
        op=model(snap.x,snap.edge_index,snap.edge_weight)
        snap_loss=loss(snap.y,op)

        #training
        model.zero_grad()

        snap_loss.backward()
        optimizer.step()

        #aggregate
        train_loss+=snap_loss.item()

    #Validation
    model.eval()
    
    with torch.no_grad():
        for t,snap in enumerate(test):
            op=model(snap.x,snap.edge_index,snap.edge_weight)
            snap_loss=loss(snap.y,op)

            test_loss+=snap_loss.item()
    
        print("Epoch: {ep}".format(ep=epoch+1))
        print("Train Loss: {l}".format(l=train_loss/(t+1)))
        print("Validation Loss: {l}".format(l=test_loss/(t+1)))


  0%|          | 0/50 [00:00<?, ?it/s]/Users/smudge/Desktop/Code/temporal-graph-seminar/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1000, 1])) that is different to the input size (torch.Size([1000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  2%|▏         | 1/50 [00:00<00:26,  1.83it/s]

Epoch: 1
Train Loss: 0.4523330176870028
Validation Loss: 0.42727333307266235


  4%|▍         | 2/50 [00:01<00:23,  2.03it/s]

Epoch: 2
Train Loss: 0.4501543072362741
Validation Loss: 0.4269075537721316


  6%|▌         | 3/50 [00:01<00:22,  2.12it/s]

...

Epoch: 48
Train Loss: 0.4497714228928089
Validation Loss: 0.4269618401924769


 98%|█████████▊| 49/50 [00:23<00:00,  2.20it/s]

Epoch: 49
Train Loss: 0.4497697991629442
Validation Loss: 0.4269651174545288


100%|██████████| 50/50 [00:23<00:00,  2.11it/s]

Epoch: 50
Train Loss: 0.4497670571009318
Validation Loss: 0.4269640423357487


In [ ]:
#Using Attention Scores
